In [52]:
import pickle as pkl
import numpy as np
from tqdm import tqdm
import pandas as pd
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments, pipeline
from nltk.tokenize import sent_tokenize, word_tokenize
import multiprocessing
import nltk
nltk.download('punkt')
import evaluate


[nltk_data] Downloading package punkt to /home/haskari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [53]:
rouge = evaluate.load('rouge')

In [54]:
sent1=["I like to move it move it"]
sent2=["We like to move it"]
res=rouge.compute(predictions=sent2, references=sent1)
res['rouge2']

0.6

In [55]:
with open('data_paraphrase/cnn.pkl', 'rb') as f:
    cnn_chat=pkl.load(f)

In [56]:
len(cnn_chat)

11490

In [57]:
cnn_chat[13]

[]

In [58]:
#number of empty summaries
count=0
bad_index=[]
for idx,item in enumerate(cnn_chat):
    if item == []:
        count+=1
        bad_index.append(idx)
print(count)



539


In [59]:
count=0
for items in cnn_chat:
    # items=nltk.sent_tokenize(items)

    if not items:
        continue
    elif len(items)==3:
        count+=1

print(count)

10945


In [60]:
print(len(cnn_chat))

11490


In [61]:
data = load_dataset("cnn_dailymail", '3.0.0')
article_key = 'article'
summary_key = 'highlights'

data=data['test']

In [62]:
data=data.select(i for i in range(len(data)) 
                 if i not in set(bad_index))

In [63]:
def tokenize(example):

    example["original_article"] = example[article_key]
    example["article"] = nltk.sent_tokenize(example[article_key])
    # if args.dataset == 'news':
    # check=example[summary_key][0]['text']
    # example["highlights"] = nltk.sent_tokenize(check)
    # else:
    example["highlights"] = nltk.sent_tokenize(example[summary_key])
        
    return example

data = data.map(tokenize, num_proc=multiprocessing.cpu_count())

Map (num_proc=48):   0%|          | 0/10951 [00:00<?, ? examples/s]

In [64]:
count=0
for summ in data['highlights']:
    count+=len(summ)

print(count)

41272


In [65]:
count=0
for summ in cnn_chat:
    count+=len(summ)

print(count)
    

32844


In [66]:
with open('data_paraphrase/xsum_capped_random.pkl', 'rb') as f:
    xsum_chat=pkl.load(f)

In [67]:
#number of empty summaries
count=0
bad_index=[]
for idx,item in enumerate(xsum_chat):
    if item == []:
        count+=1
        bad_index.append(idx)
print(count)


85


In [68]:
count=0
for items in xsum_chat:
    #items=nltk.sent_tokenize(items)
    if not items:
        continue
    elif len(items)==1:
        count+=1

print(count)

11249


In [69]:
data = load_dataset("xsum")
article_key = 'document'
summary_key = 'summary'

data=data['test']

In [70]:
data=data.select(i for i in range(len(data)) 
                 if i not in set(bad_index))

In [71]:
data = data.map(tokenize, num_proc=multiprocessing.cpu_count())

Map (num_proc=48):   0%|          | 0/11249 [00:00<?, ? examples/s]

In [72]:
count=0
for summ in data['highlights']:
    count+=len(summ)

print(count)

11250


In [73]:
count=0
for summ in xsum_chat:
    count+=len(summ)

print(count)

11249


In [74]:
with open('data_paraphrase/news_capped_random.pkl', 'rb') as f:
    news_chat=pkl.load(f)

In [75]:
#number of empty summaries
count=0
bad_index=[]
for idx,item in enumerate(news_chat):
    if item == []:
        count+=1
        bad_index.append(idx)
print(count)


7


In [76]:
count=0
for items in news_chat:
    items=items
    if not items:
        continue
    elif len(items)==1:
        count+=1

print(count)

993


In [77]:
data = load_dataset("argilla/news-summary")
article_key = 'text'
summary_key = 'prediction'
data = DatasetDict({
    'train': data['test'],
    'test': data['train']})

data=data['test']

In [78]:
data=data.select(i for i in range(len(data)) 
                 if i not in set(bad_index))

In [79]:
def tokenize_news(example):
    example["original_article"] = example[article_key]
    example["article"] = nltk.sent_tokenize(example[article_key])

    check=example[summary_key][0]['text']
    example["highlights"] = nltk.sent_tokenize(check)

    return example

In [80]:
data = data.map(tokenize_news, num_proc=multiprocessing.cpu_count())

Map (num_proc=48):   0%|          | 0/993 [00:00<?, ? examples/s]

In [81]:
count=0
for summ in data['highlights']:
    count+=len(summ)

print(count)

1005


In [82]:
count=0
for summ in news_chat:
    count+=len(summ)

print(count)

993


In [83]:
with open('data_paraphrase/reddit_capped_random.pkl', 'rb') as f:
    reddit_chat=pkl.load(f)

In [84]:
#number of empty summaries
count=0
bad_index=[]
for idx,item in enumerate(reddit_chat):
    if item == []:
        count+=1
        bad_index.append(idx)
print(count)

772


In [85]:
count=0
for items in reddit_chat:
    #items=nltk.sent_tokenize(items)
    if not items:
        continue
    elif len(items)==1:
        count+=1

print(count)

3442


In [86]:
data = load_dataset('reddit_tifu', 'long')
article_key = 'documents'
summary_key = 'tldr'
    # 80% train, 20% test + validation
train_testvalid = data['train'].train_test_split(test_size=0.2, seed=42)
# Split the 20% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)
# gather everyone if you want to have a single DatasetDict
data = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

data=data['test']

In [87]:
data=data.select(i for i in range(len(data)) 
                 if i not in set(bad_index))

In [88]:
data = data.map(tokenize, num_proc=multiprocessing.cpu_count())

Map (num_proc=48):   0%|          | 0/3442 [00:00<?, ? examples/s]

In [89]:
count=0
for summ in data['highlights']:
    count+=len(summ)

print(count)

4772


In [90]:
count=0
for summ in reddit_chat:
    count+=len(summ)

print(count)

3442
